In [2]:
import finnhub
import secrets_stock_news
import datetime

In [3]:
finnhub_client = finnhub.Client(api_key=secrets_stock_news.API_KEY)

In [4]:
str(datetime.date.today())

'2023-06-15'

In [5]:
company_news = finnhub_client.company_news('AAPL', _from=str(datetime.date.today()), to=str(datetime.date.today()))

In [6]:
general_news = finnhub_client.general_news('general', min_id=0)

In [7]:
for news in general_news:
    print(news.keys())
    break

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])


In [8]:
len(general_news)

100

In [9]:
for n in general_news:
    print(n.keys())
    break

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])


In [34]:
id = []

for news in general_news:
    id.append(news['id'])

if(len(id)):
    print(max(id))

7272796


In [ ]:
while(True):
    general_news = finnhub_client.general_news('general', min_id=0)
    sleep(60)